<a href="https://colab.research.google.com/github/AlexPazCodesUCSD/CSE151AGroupProject/blob/main/CSE151A_Group_Project_Amazon_Agent_Rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Run to install all the necessary packages for the project

%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn

In [6]:
# Loading the necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add more packages as needed

## Data Link
[Kaggle](https://www.kaggle.com/datasets/sujalsuthar/amazon-delivery-dataset)
[Github](https://github.com/AlexPazCodesUCSD/CSE151AGroupProject/blob/main/amazon_delivery.csv)

In [7]:
# Data Download into Google Colab
!wget https://raw.githubusercontent.com/AlexPazCodesUCSD/CSE151AGroupProject/main/amazon_delivery.csv

--2024-07-11 02:16:03--  https://raw.githubusercontent.com/AlexPazCodesUCSD/CSE151AGroupProject/main/amazon_delivery.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5971175 (5.7M) [text/plain]
Saving to: ‘amazon_delivery.csv.1’

amazon_delivery.csv 100%[===================>]   5.69M  --.-KB/s    in 0.09s   

2024-07-11 02:16:03 (64.7 MB/s) - ‘amazon_delivery.csv.1’ saved [5971175/5971175]



In [8]:
# Loading Data

data = pd.read_csv('amazon_delivery.csv')